In [1]:
import os

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e .

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip uninstall protobuf matplotlib -y
!pip install protobuf matplotlib==3.2

In [ ]:
!pip install pyyaml google

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
!pip list

In [ ]:
!pip install opencv-python

# Start 

In [6]:
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util, config_util
from object_detection.builders import model_builder
import tensorflow as tf
import numpy as np
import cv2 as cv
import os

In [7]:
labels = [
          {'name':'Car', 'id':1}, 
          {'name':'Bus', 'id':2},
          {'name':'Motorcycle', 'id':3},
          {'name':'Truck', 'id':4},
          {'name':'Ambulance', 'id':5}
]

with open(files['LABELMAP'], 'w+') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [8]:
# ckpt_path = paths['CHECKPOINT_PATH']
!tar -xzvf {os.path.join(paths['CHECKPOINT_PATH'],'models.tar.gz')}

x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/ckpt-21.data-00000-of-00001
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/ckpt-18.index
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/train/
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/train/events.out.tfevents.1620800318.75b522acc719.287.1556.v2
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/checkpoint
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/ckpt-17.data-00000-of-00001
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/ckpt-16.index
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/ckpt-15.data-00000-of-00001
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/ckpt-15.index
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/eval/
x Tensorflow/workspace/models/SSD_MobileNet_V2_FPNLite_320x320/eval/events.out.tfevents.162080

In [9]:
import matplotlib.pyplot as plt
import numpy as np

@tf.function
def detect_image(image):
  image, shapes = detection_model.preprocess(image)
  detections = detection_model.predict(image, shapes)
  detections_dict = detection_model.postprocess(detections,shapes)
  return detections_dict

In [10]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [12]:
# Load pipeline config and build detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoints
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'],'ckpt-21')).expect_partial()

In [20]:
# cap = cv.VideoCapture('test-data/P1033666.mp4')
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_image(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=10,
                min_score_thresh=.5,
                agnostic_mode=True,
                line_thickness=10)

    cv.imshow('object detection',  cv.resize(image_np_with_detections, (720, 480)))
    
    if cv.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv.destroyAllWindows()
        break